In [ ]:
"""
Google Colab Notebook for Processing Monster Chunks.

This script handles the large document chunks skipped by the main pipeline.
It recursively splits them into smaller pieces and generates embeddings.

Environment Variables / Secrets Required:
- DRIVE_FOLDER_NAME: The name of your DVC remote folder in Google Drive.
"""

import os
import sys

In [ ]:
# --- Helper for Secrets ---
def get_secret(name: str) -> str:
    try:
        from google.colab import userdata
        return userdata.get(name)
    except (ImportError, AttributeError, Exception):
        return os.getenv(name)

In [ ]:
# --- Configuration ---
DRIVE_FOLDER_NAME = get_secret("DRIVE_FOLDER_NAME")

if not DRIVE_FOLDER_NAME:
    print("❌ ERROR: Missing 'DRIVE_FOLDER_NAME'.")
    sys.exit(1)


In [ ]:
# --- Mount Drive ---
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# --- Dependencies ---
!pip install sentence-transformers tqdm numpy langchain

In [ ]:
# --- Setup & Logic ---
import json
import numpy as np
from sentence_transformers import SentenceTransformer
import torch
import gc
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

print("Loading BGE-M3 model...")
model = SentenceTransformer('BAAI/bge-m3', device='cuda')
print("Model loaded.")

# Paths
OUTPUT_DIR = f"/content/drive/MyDrive/{DRIVE_FOLDER_NAME}/outputs"
MONSTER_CHUNKS_FILE = os.path.join(OUTPUT_DIR, "monster_chunks_to_fix.jsonl")

if not os.path.exists(MONSTER_CHUNKS_FILE):
    raise FileNotFoundError(f"Monster chunks file not found at {MONSTER_CHUNKS_FILE}")

# Splitter Config
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]
)

MANUAL_BATCH_SIZE = 32
processed_sub_chunks = 0
total_monster_chunks_read = 0

print(f"Starting recursive processing...")

try:
    with open(MONSTER_CHUNKS_FILE, 'r', encoding='utf-8') as f:
        batch_texts = []
        batch_chunk_ids = []
        
        for line in tqdm(f, desc="Splitting monster chunks"):
            total_monster_chunks_read += 1
            try:
                data = json.loads(line)
                original_text = data.get('text', '')
                doc_id = data.get('doc_id', 'unknown')
            except json.JSONDecodeError:
                continue

            sub_chunks = text_splitter.split_text(original_text)
            
            for i, sub_text in enumerate(sub_chunks):
                sub_chunk_id = f"{doc_id}_monster_sub_{i}"
                
                # Checkpointing
                batch_num = (processed_sub_chunks) // MANUAL_BATCH_SIZE
                vector_file = os.path.join(OUTPUT_DIR, f"batch_monster_{batch_num}_vectors.npy")
                ids_file = os.path.join(OUTPUT_DIR, f"batch_monster_{batch_num}_ids.json")

                if os.path.exists(vector_file) and len(batch_texts) == 0:
                    try:
                        with open(ids_file, 'r', encoding='utf-8') as f_ids:
                            processed_sub_chunks += len(json.load(f_ids))
                    except:
                        processed_sub_chunks += MANUAL_BATCH_SIZE
                    continue
                
                batch_texts.append(sub_text)
                batch_chunk_ids.append(sub_chunk_id)

                if len(batch_texts) >= MANUAL_BATCH_SIZE:
                    gc.collect()
                    torch.cuda.empty_cache()
                    
                    batch_embeds = model.encode(batch_texts, batch_size=MANUAL_BATCH_SIZE, show_progress_bar=False)
                    
                    np.save(vector_file, batch_embeds)
                    with open(ids_file, 'w', encoding='utf-8') as f_out:
                        json.dump(batch_chunk_ids, f_out)
                    
                    tqdm.write(f"Processed monster batch {batch_num}")
                    processed_sub_chunks += len(batch_texts)
                    
                    batch_texts = []
                    batch_chunk_ids = []
                    del batch_embeds
                    torch.cuda.empty_cache()

        # Final Batch
        if batch_texts:
            print(f"Processing final monster batch...")
            batch_num = (processed_sub_chunks) // MANUAL_BATCH_SIZE
            vector_file = os.path.join(OUTPUT_DIR, f"batch_monster_{batch_num}_vectors.npy")
            ids_file = os.path.join(OUTPUT_DIR, f"batch_monster_{batch_num}_ids.json")

            if not os.path.exists(vector_file):
                gc.collect()
                torch.cuda.empty_cache()
                batch_embeds = model.encode(batch_texts, batch_size=len(batch_texts), show_progress_bar=True)
                np.save(vector_file, batch_embeds)
                with open(ids_file, 'w', encoding='utf-8') as f_out:
                    json.dump(batch_chunk_ids, f_out)
                print(f"Processed final monster batch {batch_num}")
                processed_sub_chunks += len(batch_texts)
            
    print("="*50)
    print(f"Complete. Original chunks: {total_monster_chunks_read}. New sub-chunks: {processed_sub_chunks}")
    print("="*50)

except Exception as e:
    print(f"An error occurred: {e}")
    import torch
    torch.cuda.empty_cache()